In [ ]:
"""
video: https://www.youtube.com/watch?v=vDX5kwcKmfA
1. RESEARCH AGENT LOGS INTO LINKEDIN
2. NAVIGATES TO THE PROSPECT'S PROFILE
3. ANALYZES AND SUMMARIZES THE ACTIVITIES
4. WRITER AGENT CRAFTS A TAILORED MESSAGE
"""

In [ ]:
# tools.py

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from typing_extensions import Annotated
from openai import OpenAI
import os
import time
import base64

def analyze_profile(profile_name: Annotated[str, "The name of the profile to screenshot"""]):
    filename = create_screenshot(profile_name)
    analysis = analyze_screenshot(filename)
    return analysis

def create_screenshot(profile_name):
    driver = webdriver.Chrome()
    username = os.environ.get ("LINKEDIN_EMAIL")
    password = os.environ.get ("LINKEDIN_PASSWORD")
    if not (username and password):
        raise Exception ( 'Username or password not given')
    
    driver. get ("https://www.linkedin.com/login")
    username_input = driver.find_element("id", "username")
    password_input = driver.find_element("id", "password")
    username_input.send_keys(username)
    password_input.send_keys(password)
    password_input.send_keys(Keys.RETURN) 
    driver.get(f"https://www.linkedin.com/in/{profile_name}/recent-activity/all/")
    time. sleep (3)

    screenshot_path = f"{profile_name}_linkedin.png"
    driver.save_screenshot(screenshot_path)
  
def analyse_sreenshot():
    client = OpenAI()
    base64_image = encode_image(image_path)
    response = client.chat.completions.create(
        model="gpt-40",
        messages=[
            
            {
                " role": "user",
                "content":[
                    {
                        "type": "text",
                        "text": "Describe the interests of this user as detailed as"
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data: image/jpeg; base64,{base64_image}"
                            }
                    },
                ],
            }
        ],
        max_tokens=300,
    )
    return response.choices[0].message.content



In [ ]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager, register_function
import os

In [ ]:
llm_config = {"model": "gpt-40", "api_key": os.environ ["'OPENAI_API_KEY"]}
user_proxy = UserProxyAgent("user_proxy", code_execution_config=False)

In [ ]:
linkedin_analyzer = AssistantAgent(
    name="linkedin_analyzer", 
    system_message="""
    A LinkedIn analyzer who accurately describes profiles.
    He calls the given tools to provide a description of a LinkedIn profile.
    """,
    llm_config=llm_config
)

In [ ]:
register_function(
    analyze_profile, 
    caller=linkedin_analyzer, 
    executor=user_proxy, 
    name="analyze_profiles",
    description="Returns a detailed summary of the requested LinkedIn profile.",
)

In [ ]:
writer = AssistantAgent(
    name="message_writer", 
    system_message="""
    A LinkedIn message writer named Sebastian for ai-for-devs.com.
    Its role is to write short, highly individual LinkedIn messages in the style of very experienced copywriter \
        to introduce AI consulting services based on the insights provided by the LinkedIn Analyzer.
    """,
llm_config=llm_config
)

In [ ]:
groupchat = GroupChat(
    agents=[user_proxy, 
    linkedin_analyzer, 
    writer], 
messages=[]
)

In [ ]:
user_proxy.initiate_chat(
    manager, message=f"""
    Follow these steps:
    1. The analyzer asks the user for a given LinkedIn profile.
    2. Describe the profile in a few sentences.
    3. Write an individual LinkedIn message.
    4. Ask for the next LinkedIn profile.
    """
)